# 01 · Instalación y configuración del entorno de trabajo

**Módulo I — Fundamentos**

## Objetivos:
- Verificar la instalación de Python y detectar el entorno de ejecución
- Preparar la estructura mínima del proyecto (carpetas de datos y reportes)
- Instalar y validar dependencias base para el curso
- Ejecutar una prueba rápida de importación y visualización

## Requisitos:
- Python 3.10+ o Google Colab
- Internet para instalar paquetes si hace falta

## Índice:
1. Detección de entorno (Colab/Jupyter/VSCode)
2. Preparación de carpetas
3. Instalación de dependencias
4. Verificación de versiones
5. Prueba de visualización y guardado
6. Notas para VSCode/Colab
7. Ejercicios rápidos

## 1️⃣ Detección de entorno

Identificamos si estás en Google Colab o en un Jupyter local (p.ej. VSCode).

In [ ]:
import sys, os, platform

def detectar_entorno():
    in_colab = False
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    in_vscode = bool(os.environ.get("VSCODE_PID") or os.environ.get("TERM_PROGRAM") == "vscode")

    return {
        "in_colab": in_colab,
        "in_vscode": in_vscode,
        "python_version": sys.version.replace("\n", " "),
        "executable": sys.executable,
        "platform": platform.platform(),
    }

info_entorno = detectar_entorno()
print("🔍 Información del entorno:")
for key, value in info_entorno.items():
    print(f"  {key}: {value}")

## 2️⃣ Preparación de carpetas

Crearemos la estructura estándar del proyecto:
- `data/raw` - datos originales
- `data/processed` - datos procesados
- `reports/figures` - gráficos y reportes

In [ ]:
from pathlib import Path
import json
from datetime import datetime

# Definir estructura de carpetas
BASE_DIR = Path.cwd()
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"
REPORTS_FIGURES = BASE_DIR / "reports" / "figures"

# Crear carpetas
for d in [DATA_RAW, DATA_PROCESSED, REPORTS_FIGURES]:
    d.mkdir(parents=True, exist_ok=True)

# Guardar metadatos
runtime_info = {
    "timestamp": datetime.now().isoformat(),
    "environment": info_entorno,
    "working_dir": str(BASE_DIR),
    "created_folders": [str(d.relative_to(BASE_DIR)) for d in [DATA_RAW, DATA_PROCESSED, REPORTS_FIGURES]]
}

(BASE_DIR / "runtime_info.json").write_text(json.dumps(runtime_info, indent=2), encoding="utf-8")

print("📁 Carpetas creadas:")
for d in [DATA_RAW, DATA_PROCESSED, REPORTS_FIGURES]:
    print(f"  ✅ {d.relative_to(BASE_DIR)}")
print("\n📄 Archivo generado: runtime_info.json")

## 3️⃣ Instalación de dependencias

Verificamos e instalamos los paquetes necesarios para el curso:

**Paquetes base:**
- `numpy`, `pandas` - manipulación de datos
- `matplotlib`, `seaborn` - visualización
- `openpyxl` - archivos Excel
- `lxml` - archivos XML/HTML
- `pyarrow` - archivos Parquet
- `requests` - descargas web

In [ ]:
import subprocess, importlib

PAQUETES = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "openpyxl",
    "lxml",
    "pyarrow",
    "requests",
]

def ensure_package(pkg: str):
    """Verifica si un paquete está instalado, si no lo instala"""
    try:
        importlib.import_module(pkg)
        print(f"  ✅ {pkg} - ya instalado")
        return True
    except ImportError:
        print(f"  📦 Instalando {pkg}...")
        try:
            cmd = [sys.executable, "-m", "pip", "install", "-q", "-U", pkg]
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode == 0:
                print(f"  ✅ {pkg} - instalado correctamente")
                return True
            else:
                print(f"  ❌ Error instalando {pkg}: {result.stderr}")
                return False
        except Exception as e:
            print(f"  ❌ Excepción instalando {pkg}: {e}")
            return False

print("🔧 Verificando e instalando dependencias:")
resultados = {}
for pkg in PAQUETES:
    resultados[pkg] = ensure_package(pkg)

# Resumen
exitosos = sum(resultados.values())
total = len(resultados)
print(f"\n📊 Resumen: {exitosos}/{total} paquetes instalados correctamente")

## 4️⃣ Verificación de versiones

Importamos las librerías y verificamos sus versiones para confirmar que todo funciona.

In [ ]:
print("📚 Importando librerías y verificando versiones...")

try:
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    import seaborn as sns
    import openpyxl
    import lxml
    import pyarrow
    import requests
    
    print("✅ Todas las importaciones exitosas!")
    
    versions = {
        "Python": sys.version.split()[0],
        "NumPy": np.__version__,
        "Pandas": pd.__version__,
        "Matplotlib": matplotlib.__version__,
        "Seaborn": sns.__version__,
        "OpenPyXL": openpyxl.__version__,
        "lxml": lxml.__version__,
        "PyArrow": pyarrow.__version__,
        "Requests": requests.__version__,
    }
    
    print("\n📋 Versiones instaladas:")
    for lib, version in versions.items():
        print(f"  {lib}: {version}")
        
except ImportError as e:
    print(f"❌ Error en la importación: {e}")
    print("💡 Ejecuta la celda anterior para instalar las dependencias")

## 5️⃣ Prueba de visualización y guardado

Creamos un dataset sintético, lo guardamos como CSV y generamos una visualización para probar que todo funciona correctamente.

In [ ]:
# Generar datos sintéticos
print("🎲 Generando datos sintéticos...")
rng = np.random.default_rng(seed=42)
n = 200

df = pd.DataFrame({
    "edad": rng.integers(18, 80, size=n),
    "peso": rng.normal(loc=70, scale=15, size=n),
    "altura": rng.normal(loc=170, scale=10, size=n),
    "genero": rng.choice(["M", "F"], size=n),
    "ciudad": rng.choice(["Cali", "Bogotá", "Medellín"], size=n)
})

# Calcular IMC
df["imc"] = df["peso"] / (df["altura"] / 100) ** 2

print(f"📊 Dataset creado: {df.shape[0]} filas, {df.shape[1]} columnas")
print("\n🔍 Primeras 5 filas:")
display(df.head())

# Guardar CSV
csv_path = DATA_PROCESSED / "datos_sinteticos_salud.csv"
df.to_csv(csv_path, index=False)
print(f"\n💾 CSV guardado en: {csv_path.relative_to(BASE_DIR)}")

In [ ]:
# Crear visualización
print("📈 Creando visualización...")

# Configurar estilo
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

# Crear subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle("Análisis Exploratorio - Datos Sintéticos de Salud", fontsize=16, fontweight="bold")

# Gráfico 1: Distribución de edad
sns.histplot(data=df, x="edad", hue="genero", alpha=0.7, ax=axes[0,0])
axes[0,0].set_title("Distribución de Edad por Género")

# Gráfico 2: Relación peso vs altura
sns.scatterplot(data=df, x="altura", y="peso", hue="genero", alpha=0.7, ax=axes[0,1])
axes[0,1].set_title("Relación Altura vs Peso")

# Gráfico 3: Distribución de IMC
sns.boxplot(data=df, x="ciudad", y="imc", ax=axes[1,0])
axes[1,0].set_title("Distribución de IMC por Ciudad")
axes[1,0].tick_params(axis="x", rotation=45)

# Gráfico 4: Conteo por ciudad y género
crosstab = pd.crosstab(df["ciudad"], df["genero"])
crosstab.plot(kind="bar", ax=axes[1,1])
axes[1,1].set_title("Distribución por Ciudad y Género")
axes[1,1].tick_params(axis="x", rotation=45)
axes[1,1].legend(title="Género")

plt.tight_layout()

# Guardar figura
fig_path = REPORTS_FIGURES / "analisis_exploratorio_salud.png"
plt.savefig(fig_path, dpi=300, bbox_inches="tight")
plt.show()

print(f"\n🖼️ Figura guardada en: {fig_path.relative_to(BASE_DIR)}")

## 6️⃣ Notas para VSCode y Colab

### 📝 Visual Studio Code
- **Extensiones necesarias:** Python, Jupyter
- **Seleccionar intérprete:** `Ctrl+Shift+P` → "Python: Select Interpreter"
- **Entorno virtual:** Crear con `python -m venv venv` y activar
- **Ejecución:** `Shift+Enter` para ejecutar celdas

### 🌐 Google Colab
- **Ventajas:** No requiere instalación, GPU/TPU gratuitas
- **Persistencia:** El sistema de archivos es temporal
- **Guardar trabajo:** Archivo → Guardar copia en GitHub
- **Montar Drive:** Para acceder a archivos personales

### 💡 Recomendaciones
- **Versionado:** Usar Git para control de versiones
- **Dependencias:** Documentar en `requirements.txt`
- **Estructura:** Mantener organización de carpetas consistente

## 7️⃣ Ejercicios rápidos

### 🏃‍♂️ Ejercicio 1: Instalar paquete adicional
Instala el paquete `pyyaml` usando la función `ensure_package` y verifica su versión.

### 🏃‍♀️ Ejercicio 2: Crear datos adicionales
1. Crea una carpeta `data/external/`
2. Genera un DataFrame con 10 filas de datos médicos simulados
3. Guárdalo como `pacientes_ejemplo.csv`

### 🏃‍♂️ Ejercicio 3: Personalizar visualización
1. Cambia el estilo de Seaborn a `"darkgrid"`
2. Modifica los colores del gráfico de dispersión
3. Guarda la nueva figura con un nombre diferente

### 🏃‍♀️ Ejercicio 4: Documentar versiones
Guarda el diccionario `versions` como archivo JSON en la carpeta `reports/`.

In [ ]:
# Espacio para resolver ejercicios

# Ejercicio 1: Instalar pyyaml
# ensure_package("pyyaml")

# Ejercicio 2: Crear datos adicionales
# external_dir = BASE_DIR / "data" / "external"
# external_dir.mkdir(exist_ok=True)

# Ejercicio 3: Personalizar visualización
# sns.set_style("darkgrid")

# Ejercicio 4: Guardar versiones
# versions_path = BASE_DIR / "reports" / "versions.json"

print("✍️ Completa los ejercicios editando esta celda")

## ✅ Resumen del Notebook

En este notebook hemos:

1. ✅ **Detectado el entorno** de ejecución (Colab/VSCode/Jupyter)
2. ✅ **Creado la estructura** de carpetas del proyecto
3. ✅ **Instalado dependencias** necesarias para el curso
4. ✅ **Verificado versiones** de todas las librerías
5. ✅ **Generado datos sintéticos** de ejemplo
6. ✅ **Creado visualizaciones** exploratorias
7. ✅ **Guardado archivos** CSV y figuras

### 🎯 Próximos pasos

- **Notebook 02:** Variables, estructuras de datos y funciones
- **Notebook 03:** Funciones y buenas prácticas (PEP-8)
- **Notebook 04:** Gestión de entornos virtuales

---

**💡 ¿Dudas o problemas?** Crea un Issue en el repositorio del curso.